In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('./icook_sort.csv')

In [3]:
# 對 按讚數及瀏覽數進行清洗
df['LikeStatNum'] = 0
for n ,LikeStat in enumerate(df['LikeStat']):
    LikeStat = LikeStat.replace(',', '')
    if '.' in LikeStat:
        LikeStat = int(float(LikeStat) * 10000)
    else:
        LikeStat = int(LikeStat)
    df.loc[n ,'LikeStatNum'] = LikeStat

df['PreviewNum'] = 0
for n ,Preview in enumerate(df['Preview']):
    PreviewNum = Preview.split(' ')[0].replace(',', '')
    try:
        if Preview.split(' ')[1][0] == '萬':
            PreviewNum = int(float(PreviewNum) * 10000)
    except IndexError:
        PreviewNum = int(PreviewNum)
#     print(PreviewNum)
    df.loc[n ,'PreviewNum'] = PreviewNum

df['Total'] = df['LikeStatNum'] + df['PreviewNum']

# 對按讚數及瀏覽數取前100高的值
df = df[['RecipeID', 'Total']].sort_values(by='Total', ascending=False)[:100] 

In [4]:
df.head()

,RecipeID,Total
9921,58068,2917000
15567,62718,1104000
8590,201318,1101000
20808,54846,999000
2492,82698,920004


In [52]:
# 創造100個使用者，對每篇食譜進行隨機評分
df_ALS_Dempo = pd.DataFrame(np.random.randint(0,5,[50,100]),columns=df['RecipeID'])
df_ALS_Dempo.head()

RecipeID,58068,62718,201318,54846,82698,70361,108069,131364,82424,110616,...,162981,54683,96182,132404,164231,203435,116042,185898,80063,115957
0,2,1,2,0,1,1,0,1,4,0,...,0,3,2,4,4,2,0,0,2,1
1,2,0,0,1,0,0,2,3,0,0,...,1,3,4,3,3,2,0,2,4,0
2,0,1,2,3,2,2,4,0,4,1,...,0,1,1,1,0,2,0,2,2,3
3,2,1,0,1,4,3,2,2,4,0,...,2,0,4,4,4,1,1,4,2,3
4,3,2,0,4,3,3,4,4,0,1,...,2,3,3,2,4,1,0,0,0,0


In [53]:
df_ALS_Demo=df_ALS_Dempo.reset_index()

In [54]:
df_ALS_Demo.head()

RecipeID,index,58068,62718,201318,54846,82698,70361,108069,131364,82424,...,162981,54683,96182,132404,164231,203435,116042,185898,80063,115957
0,0,2,1,2,0,1,1,0,1,4,...,0,3,2,4,4,2,0,0,2,1
1,1,2,0,0,1,0,0,2,3,0,...,1,3,4,3,3,2,0,2,4,0
2,2,0,1,2,3,2,2,4,0,4,...,0,1,1,1,0,2,0,2,2,3
3,3,2,1,0,1,4,3,2,2,4,...,2,0,4,4,4,1,1,4,2,3
4,4,3,2,0,4,3,3,4,4,0,...,2,3,3,2,4,1,0,0,0,0


In [55]:
df_ALS_Demo.columns

Index(['index',   58068,   62718,  201318,   54846,   82698,   70361,  108069,
        131364,   82424,
       ...
        162981,   54683,   96182,  132404,  164231,  203435,  116042,  185898,
         80063,  115957],
      dtype='object', name='RecipeID', length=101)

In [56]:
df_ALS_Demo = df_ALS_Dempo.stack().to_frame()

In [57]:
df_ALS_Demo

0
   RecipeID   
0  index     0
   58068     2
   62718     1
   201318    2
   54846     0
...         ..
49 203435    2
   116042    2
   185898    0
   80063     0
   115957    0

[5050 rows x 1 columns]

In [60]:
df_ALS_Demo = df_ALS_Dempo.reset_index()

In [63]:
df_ALS_Demo

,level_0,RecipeID,0
0,0,index,0
1,0,58068,2
2,0,62718,1
3,0,201318,2
4,0,54846,0
...,...,...,...
5045,49,203435,2
5046,49,116042,2
5047,49,185898,0
5048,49,80063,0


In [62]:
df_ALS_Demo.columns

Index(['level_0', 'RecipeID', 0], dtype='object')

In [66]:
df_ALS_Demo.rename(columns={'level_0':'userid', 0:'like'}, inplace = True)
df_ALS_Demo

,userid,RecipeID,like
0,0,index,0
1,0,58068,2
2,0,62718,1
3,0,201318,2
4,0,54846,0
...,...,...,...
5045,49,203435,2
5046,49,116042,2
5047,49,185898,0
5048,49,80063,0


In [72]:
# 把分數為0的視為未評分，進行filter
df_ALS_Demo_final = df_ALS_Dempo[(df_ALS_Demo['like'] != 0) & (df_ALS_Demo['RecipeID'] != 'index')] 

In [73]:
df_ALS_Demo_final

,userid,RecipeID,like
1,0,58068,2
2,0,62718,1
3,0,201318,2
5,0,82698,1
6,0,70361,1
...,...,...,...
5042,49,96182,3
5043,49,132404,2
5044,49,164231,1
5045,49,203435,2


In [75]:
df_ALS_Demo_final.to_csv('create_data.csv', index=False)